<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/Random_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [37]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Class Random Agent

In [38]:
class RandomAgent:

   def __init__(self, action_size):
      self.action_size = action_size

   def choose_action(self):
        return random.randrange(self.action_size)

# class Task

In [39]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0
        self.waiting_time = 0
        self.total_file_size = 0


# Class Workflow

In [40]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [41]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))
        stats["task.instructions_max"] = max(instructions, stats["task.instructions_max"])
        stats["task.instructions_min"] = min(instructions, stats["task.instructions_min"])

        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        numbers = np.random.randint(0, len(filtered_files), size)
        #print('numbers: ', numbers)
        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution

    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [42]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# class Device

In [43]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)
        self.dev_type = id.split('_')[0]

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


In [44]:
stats = {
    "self.cost_max" : 0, "self.cost_min":0,
    "task.instructions_max" : 0, "task.instructions_min":100000000,
    "task.cost_max" : 0, "task.cost_min":10000000,
    "task.total_transfer_files_size_max" : 0, "task.total_transfer_files_size_min":100000000,
    "task.storage_cost_max" : 0, "task.storage_cost_min":100000000,
    "self.total_delay_max" : 0, "self.total_delay_min":10000000,
    "task.execution_time_max" : 0, "task.execution_time_min":10000000,
    "task.comm_delay_max" : 0, "task.comm_delay_min":1000000,
    "d.waiting_time_max" : 0, "d.waiting_time_min":10000000,
    "task.total_files_size_max" : 0, "task.total_files_size_min":100000000
}

def format_dict_as_code(d, var_name="stats"):
    result = f"{var_name} = {{"
    items = list(d.items())
    for i in range(0, len(items), 2):
        k1, v1 = items[i]
        line = f'    "{k1}" : {v1}'
        if i + 1 < len(items):
            k2, v2 = items[i + 1]
            line += f', "{k2}":{v2},'
        result += line
    result += "}"
    return result

# class FogEnv

In [45]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }


        # Storage cost rate per GB per hour for each device type (in dollars)
        self.storage_cost_rate = {
            'iot': 0.00007,  # $ per GB per hour for IoT devices
            'fog': 0.000042,  # $ per GB per hour for Fog nodes
            'cloud': 0.000003  # $ per GB per hour for Cloud servers
        }


        self.transfer_cost_rate = {
            ('iot', 'fog'): 0.00005, ('iot', 'cloud'): 0.0002,
            ('fog', 'cloud'): 0.0001, ('fog', 'iot'): 0.00005,
            ('cloud', 'fog'): 0.0001, ('cloud', 'iot'): 0.0002,
        }


    def estimate_transfer_cost(self, task, device):
        total_transfer_cost = 0
        for file_name, file_size in task.input_files:
            src_id = self.file_origins.get(file_name, device.id)
            src_dev = self.get_device_by_id(src_id)
            if src_dev and src_dev.id != device.id:
                rate = self.transfer_cost_rate.get((src_dev.dev_type, device.dev_type), 0.0001)
                total_transfer_cost += file_size / (1024**2) * rate  # in MB
        return total_transfer_cost



    def assign_task(self, task, device):
        with device.resource.request() as req:
            yield req

            # Include tasks already running or queued
            total_instructions = sum(t.instructions for t in device.runnig_queue)  # Currently running
            total_instructions += sum(t.instructions for t in device.queue)       # Waiting

            waiting_time = (total_instructions * 6000) / device.mips

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                yield self.env.process(self.handle_file_transfer(task, file_name, file_size_bytes, device))




            # 2) Execute the task on this device
            execution_time = task.instructions * 6000/ device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)

            # 3) Calculate waiting time (difference between current time and arrival time)


            # 4) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost

            stats["task.cost_max"] = max(task.cost, stats["task.cost_max"])
            stats["task.cost_min"] = min(task.cost, stats["task.cost_min"])

            # 5) Calculate and add storage cost (including waiting time)
            storage_cost = self.calculate_storage_cost(task, device, waiting_time, execution_time)
            task.storage_cost = storage_cost
            #self.cost += storage_cost

            #self.cost+= self.estimate_transfer_cost(task, device)

            stats["self.cost_max"] = max(self.cost, stats["self.cost_max"])

            stats["task.storage_cost_max"] = max(task.storage_cost, stats["task.storage_cost_max"])
            stats["task.storage_cost_min"] = min(task.storage_cost, stats["task.storage_cost_min"])

            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time

            stats["task.execution_time_max"] = max(task.execution_time, stats["task.execution_time_max"] )
            stats["task.execution_time_min"] = max(task.execution_time, stats["task.execution_time_min"] )

            # 6) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 7) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()


    def calculate_storage_cost(self, task, device, waiting_time, execution_time):
        total_file_size = 0

        # Add the size of input files
        for file_name, file_size_bytes in task.input_files:
            total_file_size += file_size_bytes

        # Add the size of output files
        for out_file, file_size_bytes in task.output_files:
            total_file_size += file_size_bytes

        stats["task.total_files_size_max"] = max(total_file_size, stats["task.total_files_size_max"])
        stats["task.total_file_size_min"] = min(total_file_size, stats["task.total_files_size_min"])

        # Convert total size to GB
        total_file_size_gb = total_file_size / (1024 ** 3)  # Convert bytes to GB

        # Get the storage cost rate for the device type
        device_type = device.dev_type
        storage_cost_per_hour = self.storage_cost_rate.get(device_type, 0)

        # Calculate the total storage time (waiting time + execution time)
        total_storage_time = (waiting_time + execution_time)

        # Convert total time from seconds to hours
        total_time_in_hours = total_storage_time / 3600  # Convert seconds to hours


        # Calculate the storage cost for this task (based on total storage time)
        storage_cost = total_file_size_gb * storage_cost_per_hour * total_storage_time  # cost per GB per hour * time
        return storage_cost


    # Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):


        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        task.total_transfer_files_size += file_size_bytes

        stats["task.total_transfer_files_size_max"] = max(task.total_transfer_files_size, stats["task.total_transfer_files_size_max"])
        stats["task.total_transfer_files_size_min"] = min(task.total_transfer_files_size, stats["task.total_transfer_files_size_min"])

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / 1e6

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps

        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay

        stats["self.total_delay_max"] = max(self.total_delay, stats["self.total_delay_max"])
        stats["self.total_delay_min"] = min(self.total_delay, stats["self.total_delay_min"])

        stats["task.comm_delay_max"] = max(task.comm_delay,  stats["task.comm_delay_max"])
        stats["task.comm_delay_min"] = max(task.comm_delay,  stats["task.comm_delay_min"])



        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")





def process_workflow(env, workflow, fog_env, agent):
    # A list to hold all the task processes (concurrent tasks)

    while(True):
        if all([task.executed for task in workflow.tasks.values()]):
          break

        processes = []
        # Iterate through each task in the workflow
        for task in workflow.tasks.values():
            if task.executed:
                continue

            # Check if the task can be executed (all parent tasks are done)
            if all([parent.executed for parent in task.parents]) or task.parents == []:
                # Choose the action based on the current state (device selection)
                action = agent.choose_action()

                # Get the list of devices (IoT devices, fog nodes, and cloud servers)
                devices = fog_env.iot_devices + fog_env.fog_nodes + fog_env.cloud_servers
                device = devices[action]

                # Add the task to the device's queue
                device.add_task_to_queue(task)
                task.executed_on = device.id

                # Create a process for the task and add it to the processes list
                process = env.process(fog_env.assign_task(task, device))
                processes.append(process)  # Track the process

        # After all tasks are started, wait for all of them to finish
        yield env.all_of(processes)

# class NewSim



In [46]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()

    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'server_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = RandomAgent(action_size=self.num_totla_dev)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [47]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [48]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()

    # Print results for current run
    print(f"Total cost for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.fog_env.cost}")
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Results

## Cybershake - constant

In [49]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'constant', None)

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 21.525023611111077
Total time for Workflow CyberShake (constant, size=10): 212925.56881419255
Running simulation for Workflow: CyberShake, Size: 20, Distribution: constant
Total cost for Workflow CyberShake (constant, size=20): 42.288922222222105
Total time for Workflow CyberShake (constant, size=20): 231235.5096972488
Running simulation for Workflow: CyberShake, Size: 30, Distribution: constant
Total cost for Workflow CyberShake (constant, size=30): 58.620730555555525
Total time for Workflow CyberShake (constant, size=30): 251968.47627489682
Running simulation for Workflow: CyberShake, Size: 40, Distribution: constant
Total cost for Workflow CyberShake (constant, size=40): 76.610788888889
Total time for Workflow CyberShake (constant, size=40): 265578.70217786456
Running simulation for Workflow: CyberShake, Size: 50, Distribution: constant
Total cost for

In [ ]:
format_dict_as_code(stats, var_name="stats")

'stats = {    "self.cost_max" : 38712.27163848927, "self.cost_min":0,    "task.instructions_max" : 203.78, "task.instructions_min":0.34,    "task.cost_max" : 0.45284444444444444, "task.cost_min":0.0,    "task.total_transfer_files_size_max" : 0, "task.total_transfer_files_size_min":100000000,    "task.storage_cost_max" : 129.6453880033913, "task.storage_cost_min":0.0,    "self.total_delay_max" : 0, "self.total_delay_min":10000000,    "task.execution_time_max" : 2445.36, "task.execution_time_min":10000000,    "task.comm_delay_max" : 0, "task.comm_delay_min":1000000,    "d.waiting_time_max" : 0, "d.waiting_time_min":10000000,    "task.total_files_size_max" : 41510774022, "task.total_files_size_min":100000000,    "task.total_file_size_min" : 100000000}'

## Cybershake - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: uniform
numbers:  [2 3 2 3 3 1 2 3 0 3]
Total cost for Workflow CyberShake (uniform, size=10): 2543.02241201375
Total time for Workflow CyberShake (uniform, size=10): 85962.71999999999
Running simulation for Workflow: CyberShake, Size: 20, Distribution: uniform
numbers:  [3 0 1 4 2 2 2 3 2 0 4 1 1 0 0 0 4 2 4 4]
Total cost for Workflow CyberShake (uniform, size=20): 3420.0959876652405
Total time for Workflow CyberShake (uniform, size=20): 87861.0
Running simulation for Workflow: CyberShake, Size: 30, Distribution: uniform
numbers:  [2 4 3 3 2 4 0 2 2 2 1 0 2 2 1 3 2 2 0 0 0 3 4 0 2 3 2 1 3 0]
Total cost for Workflow CyberShake (uniform, size=30): 6681.806287570652
Total time for Workflow CyberShake (uniform, size=30): 111775.79999999999
Running simulation for Workflow: CyberShake, Size: 40, Distribution: uniform
numbers:  [4 4 2 1 0 4 4 3 3 3 2 4 1 2 4 1 4 2 4 4 3 1 1 4 4 0 3 1 3 4 3 0 1 1 4 4 3
 0 4 0]
Total cost for

## Montage - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: constant
Total cost for Workflow Montage (constant, size=10): 1.8213205222212563
Total time for Workflow Montage (constant, size=10): 73488.48
Running simulation for Workflow: Montage, Size: 20, Distribution: constant
Total cost for Workflow Montage (constant, size=20): 2.8452375154356586
Total time for Workflow Montage (constant, size=20): 77930.88000000005
Running simulation for Workflow: Montage, Size: 30, Distribution: constant
Total cost for Workflow Montage (constant, size=30): 4.250921610724635
Total time for Workflow Montage (constant, size=30): 87499.92000000001
Running simulation for Workflow: Montage, Size: 40, Distribution: constant
Total cost for Workflow Montage (constant, size=40): 6.372952891531288
Total time for Workflow Montage (constant, size=40): 94500.36000000006
Running simulation for Workflow: Montage, Size: 50, Distribution: constant
Total cost for Workflow Montage (constant, size=50): 7.759845191

## Montage - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: uniform
numbers:  [3 3 4 2 6 4 1 2 1 4]
Total cost for Workflow Montage (uniform, size=10): 1.3159717281574481
Total time for Workflow Montage (uniform, size=10): 80777.40000000001
Running simulation for Workflow: Montage, Size: 20, Distribution: uniform
numbers:  [0 6 5 4 0 4 0 2 0 6 4 5 1 2 4 0 6 3 3 3]
Total cost for Workflow Montage (uniform, size=20): 4.724837305318374
Total time for Workflow Montage (uniform, size=20): 85131.00000000001
Running simulation for Workflow: Montage, Size: 30, Distribution: uniform
numbers:  [6 4 2 3 6 1 0 0 4 1 2 2 1 1 2 3 6 0 3 2 6 2 0 4 2 6 6 4 2 0]
Total cost for Workflow Montage (uniform, size=30): 4.535389323983141
Total time for Workflow Montage (uniform, size=30): 82063.08000000002
Running simulation for Workflow: Montage, Size: 40, Distribution: uniform
numbers:  [5 5 1 3 0 0 5 1 3 0 4 2 2 1 4 4 3 5 3 3 1 6 6 0 1 3 1 5 5 6 6 3 0 2 4 0 2
 4 0 2]
Total cost for Workflow Montage (u

## Sipht - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: constant
Total cost for Workflow Sipht (constant, size=10): 226.70940878525093
Total time for Workflow Sipht (constant, size=10): 179475.72840000008
Running simulation for Workflow: Sipht, Size: 20, Distribution: constant
Total cost for Workflow Sipht (constant, size=20): 401.5558870449718
Total time for Workflow Sipht (constant, size=20): 297655.1363999999
Running simulation for Workflow: Sipht, Size: 30, Distribution: constant
Total cost for Workflow Sipht (constant, size=30): 651.2580703948282
Total time for Workflow Sipht (constant, size=30): 341018.2236000001
Running simulation for Workflow: Sipht, Size: 40, Distribution: constant
Total cost for Workflow Sipht (constant, size=40): 860.8435205895468
Total time for Workflow Sipht (constant, size=40): 446655.96479999984
Running simulation for Workflow: Sipht, Size: 50, Distribution: constant
Total cost for Workflow Sipht (constant, size=50): 1026.7319620603955
Total time

## Sipht - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: uniform
numbers:  [2 0 1 0 3 0 0 2 4 3]
Total cost for Workflow Sipht (uniform, size=10): 299.3915338264092
Total time for Workflow Sipht (uniform, size=10): 192985.3536
Running simulation for Workflow: Sipht, Size: 20, Distribution: uniform
numbers:  [3 0 1 1 3 3 4 0 4 4 0 3 0 3 2 1 1 1 3 2]
Total cost for Workflow Sipht (uniform, size=20): 1329.0248879305886
Total time for Workflow Sipht (uniform, size=20): 589728.4632000002
Running simulation for Workflow: Sipht, Size: 30, Distribution: uniform
numbers:  [3 3 2 3 3 0 4 0 1 2 0 4 0 2 2 2 4 0 3 2 3 3 1 2 3 4 4 0 3 3]
Total cost for Workflow Sipht (uniform, size=30): 864.9514620910531
Total time for Workflow Sipht (uniform, size=30): 432270.4476000002
Running simulation for Workflow: Sipht, Size: 40, Distribution: uniform
numbers:  [3 4 0 2 4 4 3 0 0 0 1 2 2 3 4 1 2 4 1 1 3 4 2 4 2 0 4 0 3 2 0 4 0 2 3 2 1
 1 1 0]
Total cost for Workflow Sipht (uniform, size=40): 1876.83164

## Inspiral - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: constant
Total cost for Workflow Inspiral (constant, size=10): 3.829054480726243
Total time for Workflow Inspiral (constant, size=10): 75765.60000000002
Running simulation for Workflow: Inspiral, Size: 20, Distribution: constant
Total cost for Workflow Inspiral (constant, size=20): 7.814274494044004
Total time for Workflow Inspiral (constant, size=20): 77879.75999999998
Running simulation for Workflow: Inspiral, Size: 30, Distribution: constant
Total cost for Workflow Inspiral (constant, size=30): 11.927766726407391
Total time for Workflow Inspiral (constant, size=30): 88037.28000000001
Running simulation for Workflow: Inspiral, Size: 40, Distribution: constant
Total cost for Workflow Inspiral (constant, size=40): 14.616347276799207
Total time for Workflow Inspiral (constant, size=40): 92157.72000000003
Running simulation for Workflow: Inspiral, Size: 50, Distribution: constant
Total cost for Workflow Inspiral (constant

## Inspiral - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: uniform
numbers:  [1 2 2 0 1 3 1 0 2 0]
Total cost for Workflow Inspiral (uniform, size=10): 12.836591317839257
Total time for Workflow Inspiral (uniform, size=10): 101841.59999999998
Running simulation for Workflow: Inspiral, Size: 20, Distribution: uniform
numbers:  [1 2 0 0 0 3 3 1 3 3 1 2 1 3 1 2 2 2 0 2]
Total cost for Workflow Inspiral (uniform, size=20): 19.466001345482585
Total time for Workflow Inspiral (uniform, size=20): 95768.39999999998
Running simulation for Workflow: Inspiral, Size: 30, Distribution: uniform
numbers:  [3 0 1 3 2 1 3 1 1 1 1 3 0 3 1 0 3 1 0 1 1 1 2 3 0 3 2 3 2 0]
Total cost for Workflow Inspiral (uniform, size=30): 28.107185456268
Total time for Workflow Inspiral (uniform, size=30): 104433.36
Running simulation for Workflow: Inspiral, Size: 40, Distribution: uniform
numbers:  [0 2 0 2 0 2 0 1 2 2 1 1 2 1 0 3 1 0 1 3 0 3 1 2 0 0 2 0 3 2 2 1 1 0 3 0 0
 1 2 2]
Total cost for Workflow Inspiral